In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("NVDA.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-04-13,260.549988,270.200012,257.000000,269.850006,269.566589,9960200
1,2020-04-14,275.000000,285.320007,273.890015,283.950012,283.651794,13273700
2,2020-04-15,277.200012,284.170013,275.380005,280.839996,280.545044,9153700
3,2020-04-16,287.079987,299.000000,286.549988,294.700012,294.390503,18640500
4,2020-04-17,298.000000,298.859985,286.940002,292.320007,292.013000,13478000


In [3]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,252.000000,252.000000,252.000000,252.000000,252.000000,2.520000e+02
mean,474.091389,481.779802,465.351429,473.936390,473.786216,1.052820e+07
std,89.469592,90.223370,87.656483,88.904888,88.998373,5.067685e+06
min,260.549988,270.200012,257.000000,269.510010,269.226990,2.447100e+06
25%,402.400002,413.427498,397.919998,407.182503,406.940490,6.943800e+06
50%,515.929993,522.825012,505.585006,514.309998,514.309998,9.189800e+06
75%,537.955001,544.577484,528.715011,536.367493,536.327866,1.285278e+07
max,606.840027,614.900024,598.000000,613.210022,613.210022,3.659210e+07


In [5]:
price_df = df.loc[:, ['Date', 'Adj Close']].copy()
price_df.head()

,Date,Adj Close
0,2020-04-13,269.566589
1,2020-04-14,283.651794
2,2020-04-15,280.545044
3,2020-04-16,294.390503
4,2020-04-17,292.013000


In [6]:
price_df.set_index(['Date'], inplace=True)
price_df.head()

,Adj Close
Date,
2020-04-13,269.566589
2020-04-14,283.651794
2020-04-15,280.545044
2020-04-16,294.390503
2020-04-17,292.013000


In [10]:
price_df['center'] = price_df['Adj Close'].rolling(window = 20).mean()
price_df.iloc[18:25]

,Adj Close,center
Date,,
2020-05-07,304.549805,NaN
2020-05-08,312.171783,289.701424
2020-05-11,322.281158,292.337152
2020-05-12,311.772217,293.743173
2020-05-13,310.873199,295.259581
2020-05-14,320.882660,296.584189
2020-05-15,339.273315,298.947205


In [11]:
price_df['ub'] = price_df['center'] + 2 * price_df['Adj Close'].rolling(window=20).std()
price_df['lb'] = price_df['center'] - 2 * price_df['Adj Close'].rolling(window=20).std()
price_df.iloc[18:25]

,Adj Close,center,ub,lb
Date,,,,
2020-05-07,304.549805,NaN,NaN,NaN
2020-05-08,312.171783,289.701424,310.291421,269.111426
2020-05-11,322.281158,292.337152,315.419792,269.254512
2020-05-12,311.772217,293.743173,317.994435,269.491911
2020-05-13,310.873199,295.259581,319.826760,270.692402
2020-05-14,320.882660,296.584189,323.680664,269.487714
2020-05-15,339.273315,298.947205,331.961807,265.932602


In [12]:
n = 20
sigma = 2
def bollinger_band(price_df, n, sigma):
    bb = price_df.copy()
    bb['center'] = price_df['Adj Close'].rolling(window = n).mean()
    bb['ub'] = price_df['center'] + sigma * price_df['Adj Close'].rolling(window=n).std()
    bb['lb'] = price_df['center'] - sigma * price_df['Adj Close'].rolling(window=n).std()
    return bb

bollinger = bollinger_band(price_df, n, sigma)

In [15]:
base_date = '2020-05-08'
sample = bollinger.loc[base_date:]
sample.head()

,Adj Close,center,ub,lb
Date,,,,
2020-05-08,312.171783,289.701424,310.291421,269.111426
2020-05-11,322.281158,292.337152,315.419792,269.254512
2020-05-12,311.772217,293.743173,317.994435,269.491911
2020-05-13,310.873199,295.259581,319.826760,270.692402
2020-05-14,320.882660,296.584189,323.680664,269.487714


In [16]:
book = sample[['Adj Close']].copy()
book['trade'] = ''
book

,Adj Close,trade
Date,,
2020-05-08,312.171783,
2020-05-11,322.281158,
2020-05-12,311.772217,
2020-05-13,310.873199,
2020-05-14,320.882660,
...,...,...
2021-04-06,554.460022,
2021-04-07,565.739990,
2021-04-08,572.679993,


In [17]:
def create_trade_book(sample):
    book = sample[['Adj Close']].copy()
    book['trade'] = ''
    return book